In [7]:
import numpy as np
import pandas as pd


# get data from the web

In [26]:
import pytse_client as tse

In [36]:
tickers = tse.download(symbols="all", write_to_csv=True,  adjust=True, include_jdate=True)

ConnectionError: HTTPConnectionPool(host='tsetmc.com', port=80): Read timed out.

In [38]:
t = tse.Ticker(symbol="وبملت", adjust=True)

In [40]:
t

In [34]:
tickers['وبملت']

,date,open,high,low,adjClose,value,volume,count,yesterday,close,jdate
0,2009-02-18,1050.0,1050.0,1050.0,1050.0,347393807250,330851245,800,1000.0,1050.0,1387-11-30
1,2009-02-21,1051.0,1076.0,1042.0,1050.0,352194689892,335334212,6457,1050.0,1057.0,1387-12-03
2,2009-02-22,1065.0,1074.0,1055.0,1065.0,8981292784,8435464,603,1050.0,1055.0,1387-12-04
3,2009-02-23,1066.0,1067.0,1059.0,1061.0,9090532333,8570222,937,1065.0,1060.0,1387-12-05
4,2009-02-25,1061.0,1064.0,1050.0,1058.0,7863032258,7434309,616,1061.0,1060.0,1387-12-07
...,...,...,...,...,...,...,...,...,...,...,...
2930,2023-02-28,2980.0,3050.0,2949.0,2996.0,673130269248,224704379,7316,3004.0,2990.0,1401-12-09
2931,2023-03-01,2999.0,3003.0,2906.0,2965.0,854972945939,288372496,7720,2996.0,2906.0,1401-12-10
2932,2023-03-04,2950.0,3004.0,2922.0,2975.0,941246115777,316363467,6824,2965.0,2988.0,1401-12-13
2933,2023-03-05,3000.0,3141.0,3000.0,3063.0,740412020721,241747379,6410,2975.0,3030.0,1401-12-14


In [35]:
tickers.columns

AttributeError: 'dict' object has no attribute 'columns'

In [10]:
np.unique(A['Ticker'])

array(['خودرو'], dtype=object)

In [9]:
Direction = "D:\\Data\\adjust\\"


In [10]:
Price = pd.read_csv(Direction+'data.csv')


In [12]:
Price.rename(columns = {'قيمت پاياني' : 'LastPrice', 'کد فارسي نماد' : 'PerasianNameOfCode',
                        'تاريخ ميلادي' : 'GregorianDate', 'تاريخ' : 'Date', 'قيمت پاياني روز معاملاتي قبل' : 'LastPriceOfYesterday'},
             inplace = True)

In [6]:
Price['Ratio'] =  Price['LastPrice']/ Price['LastPriceOfYesterday']
GPrice = Price
aa= Price.groupby('PerasianNameOfCode')
ab = pd.DataFrame(aa['Ratio'].cumprod())
ab = ab.reset_index()
GPrice['Cumprod'] = ab['Ratio']
aaa = Price.groupby('PerasianNameOfCode')
prod = pd.DataFrame(aaa['Ratio'].prod())
prod['LPrice'] = aaa['LastPrice'].last()
prod.rename(columns = {'Ratio':'prod'}, inplace = True)
prod = prod.reset_index()
GPrice = pd.merge(GPrice, prod, on = 'PerasianNameOfCode', how = 'left')
GPrice['Adjust'] = GPrice['LPrice']*(GPrice['Cumprod']/GPrice['prod'])
